In [3]:
import numpy as np
import sys
import os
import subprocess
import array
import cv2
import shutil
import errno
from datetime import timedelta
from PIL import Image
import pandas as pd
from sklearn.decomposition import PCA
from tmdbv3api import TMDb
from tmdbv3api import Movie
import tempfile

#Demo

In [ ]:
tmdb = TMDb()
tmdb.api_key = 'fbb8cc4b9f2b18b4a6606beeeaf2ca43'
tmdb.language = 'en'
tmdb.debug = True
movie = Movie()
#search = movie.search("The Browning Version")

#m = movie.details(search[1].id)
m = movie.details(43777)

female_actors_exist= 0
female_actors_percentage = 0

for people in m.casts.crew:
    if people.job == 'Director':
      print(people.name)
      print(people.gender)

for video in m.videos.results:
    if video.type == 'Trailer' and video.site == 'YouTube':
      print('https://www.youtube.com/watch?v=' + video.key)

cast_length = min(10, len(m.casts.cast))
for j in range(cast_length):
  if m.casts.cast[j].gender == 1:
    female_actors_exist = 1
    female_actors_percentage += 1


print(female_actors_exist, female_actors_percentage, cast_length)

Leon Ichaso
2
https://www.youtube.com/watch?v=7gnBatEfLds
0 0 8


#Statistics

In [5]:
col_names =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('/content/u.user', sep='|', names=col_names, encoding='latin-1')
users

FileNotFoundError: ignored

In [6]:
users['user_id'][users['sex'] == 'F']

NameError: ignored

In [ ]:
users['user_id'][users['sex'] == 'M']

In [ ]:
items = pd.read_csv('items_with_youtube_url.csv')
items['female_actors_percentage'] = items['female_actors_percentage']/items['cast_length']
items['director_gender'][items['director_gender'] == 1] = 'F'
items['director_gender'][items['director_gender'] == 2] = 'M'
items

In [ ]:
len(items[items['director_gender'] == 'F'])

In [ ]:
items.hist(column='female_actors_percentage')

#scrape director name, gender, female actor percentage

In [ ]:
col_names =  ['movie id', 'movie title' ,'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('/content/u.item', sep='|', names=col_names, encoding='latin-1')
items = items.dropna(subset=['release date'])
items.reset_index(drop = True, inplace = True)
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1681 entries, 0 to 1680
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie id            1681 non-null   int64  
 1   movie title         1681 non-null   object 
 2   release date        1681 non-null   object 
 3   video release date  0 non-null      float64
 4   IMDb URL            1679 non-null   object 
 5   unknown             1681 non-null   int64  
 6   Action              1681 non-null   int64  
 7   Adventure           1681 non-null   int64  
 8   Animation           1681 non-null   int64  
 9   Children's          1681 non-null   int64  
 10  Comedy              1681 non-null   int64  
 11  Crime               1681 non-null   int64  
 12  Documentary         1681 non-null   int64  
 13  Drama               1681 non-null   int64  
 14  Fantasy             1681 non-null   int64  
 15  Film-Noir           1681 non-null   int64  
 16  Horror

In [ ]:
for i in range(len(items['movie title'])):
    title = items.loc[i,'movie title'].strip()[:-7]
    title = title.split('(')[0]
    year = items.loc[i,'movie title'].strip()[-5:-1]
    year2 = items.loc[i,'release date'].strip()[-4:]
    title_list = title.split(', ')
    if len(title_list) == 2:
      title = title_list[1] + ' ' + title_list[0]
    items.loc[i,'query'] = title
    items.loc[i,'year'] = year
    items.loc[i,'year2'] = year2
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,query,year,year2
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,Toy Story,1995,1995
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,1,0,0,GoldenEye,1995,1995
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,1,0,0,Four Rooms,1995,1995
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,Get Shorty,1995,1995
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,1,0,0,Copycat,1995,1995


In [ ]:
tmdb = TMDb()
tmdb.api_key = 'fbb8cc4b9f2b18b4a6606beeeaf2ca43'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()
for i in range(len(items['movie title'])):
    items.loc[i,'director'] = None
    items.loc[i,'director_gender'] = None
    items.loc[i,'youtube_url'] = None
    items.loc[i,'female_actors_exist'] = 0
    items.loc[i,'female_actors_percentage'] = 0

    search = movie.search(items.loc[i,'query'])
    if search:
      for result in search:
        try:
          if abs(int(items.loc[i,'year']) - int(result.release_date[:4])) <=1 or \
          abs(int(items.loc[i,'year2']) - int(result.release_date[:4])) <=1:
            m = movie.details(result.id)
            break
        except:
          break
      
      for people in m.casts.crew:
        if people.job == 'Director':
          items.loc[i,'director'] = people.name
          items.loc[i,'director_gender'] = people.gender
          break

      for video in m.videos.results:
        if video.type == 'Trailer' and video.site == 'YouTube':
          items.loc[i,'youtube_url'] = 'https://www.youtube.com/watch?v=' + video.key
          break
          
      cast_length = min(10, len(m.casts.cast))
      items.loc[i,'cast_length'] = cast_length

      for j in range(cast_length):
        if m.casts.cast[j].gender == 1:
          items.loc[i,'female_actors_exist'] = 1
          items.loc[i,'female_actors_percentage'] += 1

items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Western,query,year,year2,director,director_gender,youtube_url,female_actors_exist,female_actors_percentage,cast_length
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,Toy Story,1995,1995,John Lasseter,2.0,https://www.youtube.com/watch?v=CxwTLktovTU,1.0,2.0,10.0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,GoldenEye,1995,1995,Martin Campbell,2.0,https://www.youtube.com/watch?v=_PMD6YenOEg,1.0,3.0,10.0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,Four Rooms,1995,1995,Allison Anders,1.0,https://www.youtube.com/watch?v=S_Pd2pGkq54,1.0,7.0,10.0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,Get Shorty,1995,1995,Barry Sonnenfeld,2.0,https://www.youtube.com/watch?v=fvBx0x9IJnk,1.0,2.0,10.0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,Copycat,1995,1995,Jon Amiel,2.0,https://www.youtube.com/watch?v=lsmXhM4yfU0,1.0,3.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,Mat' i syn,1997,1998,Aleksandr Sokurov,2.0,None,0.0,0.0,2.0
1677,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,B. Monkey,1998,1998,Michael Radford,2.0,https://www.youtube.com/watch?v=SLfl0RiOvEM,1.0,3.0,10.0
1678,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,Sliding Doors,1998,1998,Peter Howitt,2.0,https://www.youtube.com/watch?v=Da-Mizk86AE,1.0,5.0,10.0
1679,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,You So Crazy,1994,1994,Thomas Schlamme,2.0,https://www.youtube.com/watch?v=Nl-U_R7VEr8,0.0,0.0,1.0


In [ ]:
items.to_csv('items_with_youtube_url.csv', index = False)

# Search for youtube url in js, get the most likely url (not used)

We need to run `http-server ./` with items_original.csv in a separate window, then run `npx webpack server` with `retrieve_youtube_url` js, which will download a `items_with_youtube_url.txt `


In [ ]:
items = pd.read_json('/content/items_with_youtube_url.txt')

items = items.drop(['', 'year'], axis=1)
#items['year'] = items['year'].apply(lambda x: str(int(x)))
items.head()

ValueError: ignored

To get the most accurate result, if there's multiple outcome, we choose the second one, otherwise the first one unless there's no result

In [ ]:
for i in range(len(items['movie_id'])):
    youtube_list = items.loc[i,'youtube_url']
    try:
      items.loc[i,'clean_youtube_url'] = youtube_list[1]
    except:
      try:
        items.loc[i,'clean_youtube_url'] = youtube_list[0]
      except:
        items.loc[i,'clean_youtube_url'] = youtube_list
items.head()

KeyError: ignored

In [ ]:
items.to_csv('clean_youtube_url.csv', index = False)

# Manually filled in the blanks for films without youtube url, perform some modifications

In [52]:
items = pd.read_csv('items_with_youtube_url_filled.csv')
items.dropna(subset=['movie id'], inplace = True)
items['cast_length'][items['cast_length'].isna()] = 'Unclear'
items['cast_length'][items['cast_length'] == 0] = 'Unclear'
for index, row in items.iterrows():
  if row['cast_length'] == 'Unclear':
    items.loc[index, 'female_actors_percentage'] = 'Unclear'
    items.loc[index, 'female_actors_cutoff'] = 'Unclear'
  else:
    items.loc[index, 'female_actors_percentage'] = row['female_actors_percentage']/row['cast_length']
    items.loc[index, 'female_actors_cutoff'] = 1 if items.loc[index, 'female_actors_percentage'] >= 0.4 else 0

items['director_gender'][items['director_gender'] == 1] = 'F'
items['director_gender'][items['director_gender'] == 2] = 'M'
items['director_gender'][items['director_gender'] == 0] = 'Unclear'
items.to_csv('items_with_youtube_url_final_ver.csv', index = False)

<ipython-input-52-debfab09aa26>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items['cast_length'][items['cast_length'].isna()] = 'Unclear'
<ipython-input-52-debfab09aa26>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items['cast_length'][items['cast_length'] == 0] = 'Unclear'
<ipython-input-52-debfab09aa26>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items['director_gender'][items['director_gender'] == 1] = 'F'
<ipython-in

In [13]:
items[items['female_actors_percentage'] > 0.3]

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Western,query,year,year2,director,director_gender,youtube_url,female_actors_exist,female_actors_percentage,cast_length
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,Four Rooms,1995,1995,Allison Anders,F,https://www.youtube.com/watch?v=S_Pd2pGkq54,1.0,0.7,10.0
7,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,...,0,Babe,1995,1995,Chris Noonan,M,https://www.youtube.com/watch?v=tVxeoUtVF0o,1.0,0.5,10.0
8,9,Dead Man Walking (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,...,0,Dead Man Walking,1995,1995,Tim Robbins,M,https://www.youtube.com/watch?v=hr3ASqbap_s,1.0,0.5,10.0
12,13,Mighty Aphrodite (1995),30-Oct-1995,NaN,http://us.imdb.com/M/title-exact?Mighty%20Aphr...,0,0,0,0,0,...,0,Mighty Aphrodite,1995,1995,Woody Allen,M,https://www.youtube.com/watch?v=zLkA6SvZmZ4,1.0,0.4,10.0
14,15,Mr. Holland's Opus (1995),29-Jan-1996,NaN,http://us.imdb.com/M/title-exact?Mr.%20Holland...,0,0,0,0,0,...,0,Mr. Holland's Opus,1995,1996,Stephen Herek,M,https://www.youtube.com/watch?v=bcqvC7vZaqY,1.0,0.5,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,1669,MURDER and murder (1996),20-Jun-1997,NaN,http://us.imdb.com/M/title-exact?MURDER+and+mu...,0,0,0,0,0,...,0,MURDER and murder,1996,1997,Yvonne Rainer,F,NaN,1.0,0.4,10.0
1670,1672,Kika (1993),01-Jan-1993,NaN,http://us.imdb.com/M/title-exact?Kika%20(1993),0,0,0,0,0,...,0,Kika,1993,1993,Pedro Almodóvar,M,https://www.youtube.com/watch?v=z5bmEnGrA4s,1.0,0.5,10.0
1673,1675,"Sunchaser, The (1996)",25-Oct-1996,NaN,"http://us.imdb.com/M/title-exact?Sunchaser,%20...",0,0,0,0,0,...,0,The Sunchaser,1996,1996,Michael Cimino,M,https://www.youtube.com/watch?v=BFG7Cd9iClc,1.0,0.4,10.0
1674,1676,"War at Home, The (1996)",01-Jan-1996,NaN,http://us.imdb.com/M/title-exact?War%20at%20Ho...,0,0,0,0,0,...,0,The War at Home,1996,1996,Emilio Estevez,M,https://www.youtube.com/watch?v=x2sjkcs6F_w,1.0,0.4,10.0


In [63]:
len(items[items['director_gender'] == 'F'])/len(items)

0.07019631171921475

In [62]:
len(items[items['director_gender'] == 'Unclear'])

76

In [ ]:
items.hist(column='female_actors_percentage')

# Extract frames from videos

In [ ]:
SAVING_FRAMES_PER_SECOND = 3
SAVING_SECONDS = 1080
DIM = (299, 299)

In [ ]:
def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05)
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")

def get_saving_frames_durations(cap, saving_fps, saving_seconds):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    if cap.get(cv2.CAP_PROP_FPS) != 0:
      clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    else:
      clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / 24
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
      if len(s) < saving_seconds:
        s.append(i)
    return s, clip_duration

def extract_frames(video_file):
    filename, _ = os.path.splitext(video_file)
    filename += ' frames'
    # make a folder by the name of the video file
    if not os.path.isdir(filename):
        os.mkdir(filename)
    # read the video file
    cap = cv2.VideoCapture(video_file)
    # get the FPS of the video
    fps = 24
    if cap.get(cv2.CAP_PROP_FPS) != 0:
      fps = cap.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    #saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # get the list of duration spots to save
    saving_frames_durations, clip_duration = get_saving_frames_durations(cap, SAVING_FRAMES_PER_SECOND, SAVING_SECONDS)
    # start the loop
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration,
            # then save the frame
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            resized_frame = cv2.resize(frame, dsize = DIM, interpolation = cv2.INTER_AREA)
            cv2.imwrite(os.path.join(filename, f"frame{frame_duration_formatted}.jpg"), resized_frame)
            
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1
    return clip_duration

# Run Inception on frames, get embedding

In [ ]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [ ]:
def run_inception(folder_name):
    layer_name = 'avg_pool'
    model = tf.keras.applications.inception_v3.InceptionV3(weights="imagenet")
    model_before_classification = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
    #print(model.summary())
    dataset = tf.keras.preprocessing.image_dataset_from_directory(folder_name, \
    labels=None, image_size=(299, 299))
    dataset = dataset.map(lambda x : (x / 255.0 - 1.0)*2.0)

    embeddings = model_before_classification.predict(dataset, verbose=0)

    #how to aggregate frame embedding into video embedding?
    return np.mean(embeddings, axis=0)

In [ ]:
#random initialize vs video embedding, movie wise feature , feature to debias,
#data analysis, rating, Neural recommender, vs recommendtion result

In [ ]:
#tqdm time

In [ ]:
'''
col_names =  ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('/content/u.data', sep='\t', names=col_names, encoding='latin-1')
ratings = ratings.sort_values(by=['user_id'])
ratings
'''

,user_id,movie_id,rating,unix_timestamp
66567,1,55,5,875072688
62820,1,203,4,878542231
10207,1,183,5,875072262
9971,1,150,5,876892196
22496,1,68,4,875072688
...,...,...,...,...
96823,943,427,4,888639147
70902,943,12,5,888639093
84518,943,284,2,875502192
72321,943,62,3,888640003


In [ ]:
#ratings[ratings['user_id'].isin(users['user_id'][users['sex'] == 'F'])]

,user_id,movie_id,rating,unix_timestamp
9201,2,303,4,888550774
12150,2,316,5,888979693
52964,2,237,4,888552017
20534,2,282,4,888551922
11989,2,257,4,888551062
...,...,...,...,...
73676,942,479,4,891283118
67222,942,604,4,891283139
95675,942,478,5,891283017
85822,942,659,5,891283161


In [ ]:
'''
users = pd.DataFrame(np.zeros((max(user_data['user_id'])+1, max(user_data['movie_id'])+1)))
users = users.drop(0, axis = 0)
users = users.drop(0, axis = 1)
for i in user_data['user_id'].unique():
  for j in user_data['movie_id'][user_data['user_id'] == i].unique():
    users.loc[i,j] = 1
users
'''

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
942,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# For each film, download trailer, run through the embedding pipeline

In [ ]:
download = pd.read_csv('items_with_youtube_url_filled.csv')
download = download[download['youtube_url'].notna()]
download

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Western,query,year,year2,director,director_gender,youtube_url,female_actors_exist,female_actors_percentage,cast_length
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,Toy Story,1995,1995,John Lasseter,2.0,https://www.youtube.com/watch?v=CxwTLktovTU,1.0,2.0,10.0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,GoldenEye,1995,1995,Martin Campbell,2.0,https://www.youtube.com/watch?v=_PMD6YenOEg,1.0,3.0,10.0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,Four Rooms,1995,1995,Allison Anders,1.0,https://www.youtube.com/watch?v=S_Pd2pGkq54,1.0,7.0,10.0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,Get Shorty,1995,1995,Barry Sonnenfeld,2.0,https://www.youtube.com/watch?v=fvBx0x9IJnk,1.0,2.0,10.0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,Copycat,1995,1995,Jon Amiel,2.0,https://www.youtube.com/watch?v=lsmXhM4yfU0,1.0,3.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,Mat' i syn,1997,1998,Aleksandr Sokurov,2.0,https://www.youtube.com/watch?v=E7GRtbW64SM,0.0,0.0,2.0
1677,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,B. Monkey,1998,1998,Michael Radford,2.0,https://www.youtube.com/watch?v=SLfl0RiOvEM,1.0,3.0,10.0
1678,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,Sliding Doors,1998,1998,Peter Howitt,2.0,https://www.youtube.com/watch?v=Da-Mizk86AE,1.0,5.0,10.0
1679,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,You So Crazy,1994,1994,Thomas Schlamme,2.0,https://www.youtube.com/watch?v=Nl-U_R7VEr8,0.0,0.0,1.0


In [ ]:
os.chdir('/content/')

def pipeline(df, output_name):
    movie_index = []
    clip_durations = []
    embeddings = []

    #for each film
    for index, film in df.iterrows():

        #make temp directory
        filename = film['movie title']
        if not os.path.isdir(filename):
            os.mkdir(filename)
        filedir = '/content/{}/'.format(filename)
        #download video
        try:
          youtube_url = film['youtube_url']
          file_savename = '{}.%(ext)s'.format(filename)
          #--download-sections "*00:00-06:00"
          !yt-dlp -f 'bv*[height<=720]' -o "$file_savename" -P "$filedir" \
          --max-filesize 100m $youtube_url

          #extract frames and save duration
          clip_duration = extract_frames(filedir + os.listdir(filedir)[0])

          #convert images to embedding, save to list
          embedding = run_inception(filedir + '{} frames'.format(filename))

          #save clip duration and embedding
          movie_index.append(index)
          clip_durations.append(clip_duration)
          embeddings.append(embedding)

          #delete directory
          shutil.rmtree(filedir)
          !yt-dlp --rm-cache-dir
        except:
          continue

    output = np.array([movie_index, clip_durations, embeddings])
    np.save(output_name, output, allow_pickle=True)

    return output

In [ ]:
parts = np.array_split(download, 50)
for i in range(1):
    pipeline(parts[i], str(i))

[youtube] CxwTLktovTU: Downloading webpage
[youtube] CxwTLktovTU: Downloading android player API JSON
[info] CxwTLktovTU: Downloading 1 format(s): 247
[download] Destination: /content/Toy Story (1995)/Toy Story (1995).webm
[download] 100% of   11.71MiB in 00:00:00 at 31.83MiB/s
Found 271 files belonging to 1 classes.
Removing cache dir /root/.cache/yt-dlp ..
[youtube] _PMD6YenOEg: Downloading webpage
[youtube] _PMD6YenOEg: Downloading android player API JSON
[info] _PMD6YenOEg: Downloading 1 format(s): 247
[download] Destination: /content/GoldenEye (1995)/GoldenEye (1995).webm
[download] 100% of   15.00MiB in 00:00:00 at 35.50MiB/s
Found 513 files belonging to 1 classes.
Removing cache dir /root/.cache/yt-dlp ..
[youtube] S_Pd2pGkq54: Downloading webpage
[youtube] S_Pd2pGkq54: Downloading android player API JSON
[youtube] S_Pd2pGkq54: Downloading tv embedded player API JSON
[youtube] S_Pd2pGkq54: Downloading web embedded client config
[youtube] S_Pd2pGkq54: Downloading player 72d3c60a


<ipython-input-14-91bc23a4e033>:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  output = np.array([movie_index, clip_durations, embeddings])


In [ ]:
with open('0.npy', 'rb') as f:
    a = np.load(f, allow_pickle=True)
    print(a)

[[0 1 2 4 5 6 7 10 11 12 13 15 16 17 18 19 20 21 22 24 25 26 27 28 29 30
  31 32]
 [90.17341666666665 171.00416666666666 0.0 119.994875 88.96387499999999
  0.0 127.79166666666667 143.04166666666666 144.60279166666666
  27.994633333333333 85.21012499999999 80.04663333333333 86.7
  103.39495833333332 97.30554166666666 159.7012083333333 116.0
  174.50766666666667 77.54166666666667 138.22141666666667 114.989875
  158.96 153.11129166666666 202.26666666666668 98.04166666666667
  144.0605833333333 131.506375 94.97472673128249]
 [array([0.39485422, 0.31839377, 0.01428923, ..., 0.36252853, 0.29716945,
         0.28621638], dtype=float32)
  array([0.28243566, 0.2777684 , 0.01331601, ..., 0.3691073 , 0.30552626,
         0.24498859], dtype=float32)
  array([0.38407522, 0.29742864, 0.03081945, ..., 0.34464055, 0.32826343,
         0.21584655], dtype=float32)
  array([0.43229693, 0.2899653 , 0.01148118, ..., 0.42627874, 0.35532087,
         0.2659685 ], dtype=float32)
  array([0.23504393, 0.2394197

In [ ]:
pca = PCA(n_components=1500, whiten=True)
principalComponents = pca.fit_transform(embeddings)

NameError: ignored